# Introductory EEG Analysis with SciPy/Python 3

I use functions from the Scipy Signal Processing module that sweep a lot of the mathematical details under the rug and let us look at the end-to-end procedure.
Most functions that I call are intuitively named and interpretable by a student that has done any kind of signal collection class or lab.
I do not use any of the EEG-specific toolboxes that exist for the field standard languages (primarily MATLAB, and to a lesser extent Python).
These packages are far more powerful and are set up to manage vendor-specific data formats, but can be hard to approach for a newcomer.

This pipeline is based on a 3-channel recording of a basic auditory oddball paradigm designed to elicit a P300 ERP. 
We know the channel order in the data file:
1. TTL
2. V EOG (V EEG technically, single electrode)
3. H EOG (H EEG technically, single electrode)
4. Fz
5. Cz
6. Pz
7. Some other columns that recorded by the script that have no consequence here.

This script can be adapted to more channels in the initial assignment of `eeg` array.

In [ ]:
#%matplotlib notebook
import scipy.signal as sig
import matplotlib.pyplot as plt
import numpy as np

## User tunable parameters

Set the paths to point to your data files.
Replace all `None` with values.
Adjust the processing parameters and see how they affect the final results.

In [ ]:
# File paths
data_file       = '/path/to/data.ext'
sequence_file   = '/path/to/aux/files.ext'

# Data info
sample_rate     = None # Hertz
nyquist         = 0.5 * sample_rate
stim_latency    = None # samples - corresponds to our hardware (difference between TTL and soundcard)

# Filtering: Butterworth bandpass
hpf_freq        = None # Hertz
lpf_freq        = None # Hertz
filter_order    = None 

# Filtering: powerline notch
plnotch_freq    = None # Hertz
plnotch_bw      = None # Hertz

# Peak detection
d_ttl_threshold = None # microVolts per second
blink_threshold = None # microVolts
blink_width     = None # samples between blinks

# Epoch window definition
pre_stim_time   = None # seconds
post_stim_time  = None # seconds
blink_buf_time  = None # seconds outside of epoch to match blinks
blink_buffer    = sample_rate * blink_buf_time

## Initial data import and exploration

In [ ]:
# Import the data from file
data = np.loadtxt(data_file, dtype='double')

number_of_samples = data.shape[0]

# Plot the raw data - columns of the array vs. row index
index = range(number_of_samples)
number_of_subplots = data.shape[1]

plt.figure(figsize=[10,20])

for i in range(number_of_subplots):
    ax1 = plt.subplot(number_of_subplots, 1, i+1)
    ax1.plot(index,data[:,i])

plt.subplots_adjust(hspace=0.5)
plt.show()

From the above plot can you visually identify any of the traces?

We can sort our single data array into seperate arrays for readability.
Compare this to your notes from collecting data.
You'll need to identify the correct slices of the `data` array that correspond to each type of data.

In [ ]:
ttl   = data[:, 0]
veog  = data[:, 1]
eeg   = data[:, 3:6]
number_of_channels = eeg.shape[-1]

Before you charge ahead with analysis, it is good to look the frequency spectra of the main channel data.
This can identify noise or bad data.

Use `scipy.signal.periodogram` to make a power spectral density (PSD) plot.

In [ ]:
(freqs, eeg_spectra) = sig.periodogram(eeg, sample_rate, return_onesided=True, axis=0)

# Plot spectra
fig = plt.figure(figsize=[10,10])

for i in range(number_of_channels):
    ax1 = plt.subplot(number_of_channels, 1, i+1)
    ax1.loglog(freqs, eeg_spectra[:,i])
    ax1.set_ylim(1e-2, 1e13)
    
fig.text(0.50, 0.04, 'Frequency (Hz)', ha='center', fontsize=20)
fig.text(0.04, 0.50, r'Power Spectral Density (V$^2$/Hz)', ha='center', va='center', rotation='vertical', fontsize=20)
plt.subplots_adjust(hspace=0.25)
plt.show()

Do you have 60 Hz line noise contaminating the data?

## Temporal filtering

We will filter the data while it is in continuous form to minimize edge ringing.

First, design our bandpass and notch filters and apply them only to the EEG data channels.

In [ ]:
# Design the high-pass and low-pass filter parameters
low        = hpf_freq / nyquist
high       = lpf_freq / nyquist
(b, a)     = sig.butter(filter_order, [low, high], btype='band')

# Apply filters using forward/reverse zero-phase method filtfilt
eeg_bpfilt = sig.filtfilt(b, a, eeg, axis=0)

In [ ]:
# Design IIR notch for the powerline filter
w0           = plnotch_freq / nyquist
Q            = w0 / plnotch_bw
(b, a)       = sig.iirnotch(w0, Q)
eeg_bpplfilt = sig.filtfilt(b, a, eeg_bpfilt, axis=0)

Bravely ignore the future deprecation warningg and proceed to check your work.

In [ ]:
(freqs, eeg_filt_spectra) = sig.periodogram(eeg_bpplfilt, sample_rate, return_onesided=True, axis=0)

# Plot spectra
fig = plt.figure(figsize=[10,10])

for i in range(number_of_channels):
    ax1 = plt.subplot(number_of_channels, 1, i+1)
    ax1.loglog(freqs, eeg_spectra[:,i])
    ax1.loglog(freqs, eeg_filt_spectra[:,i])
    ax1.set_ylim(1e-2, 1e13)
    
fig.text(0.50, 0.04, 'Frequency (Hz)', ha='center', fontsize=20)
fig.text(0.04, 0.50, r'Power Spectral Density (V$^2$/Hz)', ha='center', va='center', rotation='vertical', fontsize=20)
plt.subplots_adjust(hspace=0.25)
plt.show()

On a periodogram (PSD plot) we can see the filter corners. If you bring your lowpass corner low enough, the powerline notch may not even be necessary. 

Look at the time domain as well.

In [ ]:
fig = plt.figure(figsize=[10,10])

for i in range(number_of_channels):
    ax1 = plt.subplot(number_of_channels, 1, i+1)
    ax1.plot(index, eeg[:,i]-np.mean(eeg[:,i]))
    ax1.plot(index, eeg_bpplfilt[:,i])
    
fig.text(0.50, 0.04, 'Sample', ha='center', fontsize=20)
fig.text(0.04, 0.50, r'EEG Signal ($\mu$V)', ha='center', va='center', rotation='vertical', fontsize=20)
plt.subplots_adjust(hspace=0.25)
plt.show()

A de-meaned copy of original signals to show them in the same plot.
These might affect the y axis limits, so remove these to see the new data clearly.

In [ ]:
fig = plt.figure(figsize=[10,10])

for i in range(number_of_channels):
    ax1 = plt.subplot(number_of_channels, 1, i+1)
    ax1.plot(index, eeg_bpplfilt[:,i])

fig.text(0.50, 0.04, 'Sample', ha='center', fontsize=20)
fig.text(0.04, 0.50, r'EEG Signal ($\mu$V)', ha='center', va='center', rotation='vertical', fontsize=20)
plt.subplots_adjust(hspace=0.25)
plt.show()

Are the data aren't intelligible yet? 
What can you identify in the filtered data?

Now the temporal filtering is finished for the continuous data. 
Next, prepare for epoching our data into trials around stimuli.

## Locating trials with the TTL (Stim) channel data

The TTL channel is a record of when the computer sent stimulus to the participant.
You need to detect the leading edges of the square pulses to identify the samples that mark auditory stimuli.

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(index, ttl)
plt.ylabel(r'TTL signal ($\mu$V)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

Take the discrete derivative using `numpy.diff` and locate the maxima. 
The falling edges will be minima.
The baseline is not a problem in this approach, so filtering is not necessary.

In [ ]:
d_ttl = np.diff(ttl, axis=0)

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(d_ttl)
plt.ylabel(r'dTTL ($\mu$V/s)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

The baseline should be flat, and the train of impulses can be marked.

Next, `scipy.signal.find_peaks` does exactly what it says on the tin.

In [ ]:
(peaks, properties) = sig.find_peaks(d_ttl, height=d_ttl_threshold)

fig = plt.figure(figsize=[10,5])
plt.plot(d_ttl)
plt.scatter(peaks, properties['peak_heights'], s=50, c='red', marker='o')
plt.ylabel(r'dTTL ($\mu$V/s)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

**Zoom and enhance.**

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(d_ttl)
plt.scatter(peaks, properties['peak_heights'], s=50, c='red', marker='o')
plt.ylabel(r'dTTL ($\mu$V/s)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.xlim(left=1000, right=10000)
plt.show()

The values you care about are the sample numbers, which are contained in `peaks`.

### Compensate for hardware latency

If you are properly careful you would have measured the latency in your EEG system between the TTL pulse edge and the stimulus delivery.
You can offset the values of the `peaks` array to compensate.

In [ ]:
peaks = peaks - stim_latency

You can check that these values line up with the leading edges of the pulse.
Bring the pre-stim baseline to around zero to illustrate.

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(index, ttl - np.mean(ttl[:3000]))
plt.scatter(peaks, np.zeros(peaks.shape), s=50, c='red', marker='o')
plt.ylabel(r'(shifted) TTL signal ($\mu$V)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.xlim(left=1000, right=10000)
plt.show()

### Define trials using triggers and processing parameters

Now that the stimuli samples are known, we need to identify the surrounding temporal regions of interest within the continuous data streams.
Derive epochs by anchoring around the trigger markers, and extending some specificed time before and after.

In [ ]:
epochs = np.array([peaks - pre_stim_time * sample_rate, peaks + post_stim_time * sample_rate ])
epochs = epochs.T

number_of_trials = epochs.shape[0]

print(epochs.shape)
#print(epochs[:11,:])

200 trials, just as planned.
The array contains the start and end sample labels of each epoch.

The trials need to be sorted according to their condition.

## Identifying standard and deviant trials using a sequence

We have a sequence file that labels the deviant tones that should contain the P300.
This information will be used when averaging trials by condition.
Note that I subtract 1 from the list, element-wise, because the list counts from 1, whereas Python is counts from 0.

In [ ]:
deviant_seq = np.loadtxt(sequence_file, delimiter=',', dtype='int')
deviant_seq -= 1
print(deviant_seq)

For convenience, we can store this list inside the epoch_samples array.
Create an extra column to mark blinks/artifacts trials.

In [ ]:
tmp = np.zeros([epochs.shape[0], epochs.shape[1] + 2], dtype='int')
tmp[:,:2] = epochs
tmp[deviant_seq, 2] = 1
epochs = tmp

print(epochs[:11, :])

## Finding bad trials from blinks in V EOG

Apply a high-pass filter to V EOG in order to straighten out the baseline.
Use peak detection in V EOG to find artifacts.

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(index, veog)
plt.ylabel(r'Raw V EOG signal ($\mu$V)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

It is possible to detect peaks from a signal with drifting baseline, but we can also deal with it via a HPF.

In [ ]:
# Design the high-pass filter parameters
low       = hpf_freq / nyquist
(b, a)    = sig.butter(filter_order, low, btype='highpass')
veog_filt = sig.filtfilt(b, a, veog, axis=0)

In [ ]:
fig = plt.figure(figsize=[10,5])
plt.plot(index, veog - np.mean(veog))
plt.plot(index, veog_filt)
plt.ylabel(r'Filtered V EOG ($\mu$V)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

De-meaned the original signal for clarity.
Compare the baselines, and make sure the blink peaks are intact.

In [ ]:
(blinks, properties) = sig.find_peaks(veog_filt, height=blink_threshold, distance=blink_width)

fig = plt.figure(figsize=[10,5])
plt.plot(index, veog_filt)
plt.scatter(blinks, properties['peak_heights'], s=50, c='red', marker='o')
plt.ylabel(r'V EOG ($\mu$V)', fontsize=20)
plt.xlabel('Sample', fontsize=20)
plt.show()

There are many blinks which is not ideal.
Note that I had to tune the `find_peaks` parameters from its last use in triggers.
We must return to our `epochs` and give them the bad news.

In [ ]:
reject_standard = 0
reject_deviant = 0

for i in range(number_of_trials):
    
    if any(np.logical_and(blinks>=(epochs[i,0]-blink_buffer),   # For each trial, is there a blink in its
                          blinks<=(epochs[i,1]+blink_buffer))): # sample span +/- buffer region
        epochs[i, -1] = 1            # Mark this trial with a 1 to indicate blink is present
    
        if epochs[i, -2] == 1:       # Count how many trials we are losing of each kind
            reject_deviant  += 1

        elif epochs[i, -2] == 0:
            reject_standard += 1
            
    else: # If you get your logic wrong, this lets it clean itself on re-run
        epochs[i, -1] = 0

print('Number of blinks/artifacts: %u' % blinks.shape)
print('Standard trials lost to blinks: %u' % reject_standard)
print('Deviant trials lost to blinks: %u' % reject_deviant)
print('\nBlinks:')
print(blinks)
print('\n\tStart\tStop\tTrial\tBad')
print(epochs[:21, :])
print('[...]')

How many trials did you lose to artifacts?
How many do you have left of each conditions?

## Segment data into epochs and average like-trials

Using our epochs array that contains the time, type, and rejection of every trial, we can reshape the `eeg_bpplfilt` array into trials.

Currently, the shape of the eeg data is samples-by-channels.
First, seperate this into (samples per trial)-by-channels-by-(number of trials), then seperate into standard and deviant.
During the process, subtract the mean of the pre-stimulus portion of the epoch window out of each epoch.
This brings every trial to a similar baseline to prevent issues with averaging.
With blinks recorded in the same array, check each trial as we go in case it needs to be ignored by the average.

In [ ]:
epoch_window    = int(sample_rate * (pre_stim_time + post_stim_time)) + 1 # Include the peak sample
baseline_window = int(sample_rate * pre_stim_time)                        # both in units of samples

standard_trial  = np.zeros([epoch_window, number_of_channels], dtype='double')
deviant_trial   = np.zeros([epoch_window, number_of_channels], dtype='double')
tmp             = np.zeros([epoch_window, number_of_channels], dtype='double')

keep_standard   = 0
keep_deviant    = 0

for i in range(number_of_trials):
    
    if epochs[i, -1] == 1: # Check if bad trial up front in the loop, as it skips the rest if true
        continue
        
    tmp = eeg_bpplfilt[epochs[i,0]:epochs[i,1], :]  # Select data
    tmp -= np.mean(tmp[:baseline_window,:], axis=0) # Subtract mean of baseline
    
    if epochs[i, -2] == 0: # If standard, do this
        standard_trial += tmp
        keep_standard  += 1
    
    if epochs[i, -2] == 1: # If deviant, do this
        deviant_trial  += tmp
        keep_deviant   += 1
        
standard_trial /= keep_standard # Divide to complete averages
deviant_trial  /= keep_deviant

print('Standard trials kept: %u' % keep_standard)
print('Deviant trials kept: %u' % keep_deviant)

That previous step was terse.
The whole anaylsis came together in just a few lines of code.
The `for` loop iterated through all trials to do the following:
1. Check if this trial was flagged for containing an artifact or blink (if true skip to next trial),
1. Subtract the average of the baseline pre-stimulus signal from each trial (each channel seperately),
1. Sort trial into standard or deviant pools, 
1. Sum trial into the correct pool,
1. Keep track of how many trials were summed into each pool.

After the loop, each trial summation is divided by the number of trials that were summed into it, this is the average.


In [ ]:
time_axis = ( np.arange(epoch_window) / sample_rate - pre_stim_time ) * 1000 # Convert seconds to ms

fig = plt.figure(figsize=[10,10])

for i in range(number_of_channels):
    ax1 = plt.subplot(number_of_channels, 1, i+1)
    ax1.plot(time_axis, standard_trial[:,i]/1000, label='Standard')  # Convert uV to mV for readability
    ax1.plot(time_axis, deviant_trial[:,i]/1000, label='Deviant')
    ax1.set_ylim(-10, 10)
    ax1.grid()
    ax1.legend()
    
fig.text(0.50, 0.04, 'Time, stimulus relative (ms)', ha='center', fontsize=20)
fig.text(0.04, 0.50, r'Trial averaged EEG signal (mV)', ha='center', va='center', rotation='vertical', fontsize=20)
fig.text(0.14, 0.85, 'Electrode: Fz', ha='left', fontsize=20)
fig.text(0.14, 0.58, 'Electrode: Cz', ha='left', fontsize=20)
fig.text(0.14, 0.31, 'Electrode: Pz', ha='left', fontsize=20)
plt.subplots_adjust(hspace = 0.25)
plt.show()

Both standard and deviants should have a positive peak at around 100 ms, the N100.
This indicates the participant was hearing something.
Do you see an expanded negative deflecton in the 250-300 ms window of the deviant traces as compared to the standards?
This is the P300.

Note that the polarity of the peak label (P/N) corresponds physiological excitation or inhibition, not voltage.